In [3]:
import networkx as nx
import os
import pybel
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from se_kge.get_url_requests import cid_to_synonyms, get_gene_names
from se_kge.graph_preprocessing import get_drugbank_graph, get_sider_graph, combine_pubchem_drugbank

# Building SIDER and DrugBank graphs

In [2]:
sider_graph = get_sider_graph()

indications: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 184764/184764 [01:58<00:00, 1557.10it/s]


In [3]:
drugbank_graph = get_drugbank_graph()

Mapping drug-protein interactions to BEL: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 116911/116911 [06:45<00:00, 288.04it/s]


In [4]:
sider_side_effects = 0
sider_drugs = 0
for node in sider_graph.nodes():
    if node.namespace == 'umls':
        sider_side_effects+=1
    else:
        sider_drugs+=1
print('SIDER graph has a total of %d nodes' % len(sider_graph.nodes()))
print('The number of Side Effects nodes is %d' % sider_side_effects)
print('The number of Drugs nodes is %d' % sider_drugs)

SIDER graph has a total of 8497 nodes
The number of Side Effects nodes is 6990
The number of Drugs nodes is 1507


In [5]:
drugbank_proteins = 0
drugbank_drugs = 0
for node in drugbank_graph.nodes():
    if node.namespace == 'uniprot':
        drugbank_proteins+=1
    else:
        drugbank_drugs+=1
print('DrugBank Graph has a total of %d nodes' % len(drugbank_graph.nodes()))
print('The number of Proteins nodes is %d' % drugbank_proteins)
print('The number of drugs nodes is %d' % drugbank_drugs)

DrugBank Graph has a total of 8059 nodes
The number of Proteins nodes is 2781
The number of drugs nodes is 5278


In [6]:
pybel.to_pickle(sider_graph, os.path.join(os.pardir, "resources", "basic_graphs", "sider_graph.pickle"))

In [7]:
pybel.to_pickle(drugbank_graph, os.path.join(os.pardir, "resources", "basic_graphs", "drugbank_graph.pickle"))

# Combining both graphs into a complete graph

In [4]:
sider_graph = pybel.from_pickle(os.path.join(os.pardir, "resources", "basic_graphs", "sider_graph.pickle"))

In [5]:
drugbank_graph = pybel.from_pickle(os.path.join(os.pardir, "resources", "basic_graphs", "drugbank_graph.pickle"))

In [6]:
drugbank_pubchem_mapping = os.path.join(os.pardir, "resources", "mapping", "drugbank_pubchem_mapping.tsv")

In [7]:
full_graph = combine_pubchem_drugbank(drugbank_pubchem_mapping, drugbank_graph, sider_graph)

create pubchem-drugbank mapping dictionary: 8540it [00:04, 1888.63it/s]
Removing nodes that were not relabeled: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 770/770 [00:00<00:00, 5079.32it/s]


In [8]:
proteins = 0
side_effects = 0
drugs = 0
for node in full_graph.nodes():
    if node.namespace == 'uniprot':
        proteins+=1
    elif node.namespace == 'umls':
        side_effects+=1
    else:
        drugs+=1
print('The combined graph has a total of %d nodes' % len(full_graph.nodes()))
print('The number of Proteins nodes is %d' % proteins)
print('The number of Side Effects nodes is %d' % side_effects)
print('The number of Drugs nodes is %d' % drugs)

The combined graph has a total of 15024 nodes
The number of Proteins nodes is 2596
The number of Side Effects nodes is 6990
The number of Drugs nodes is 5438


In [9]:
pybel.to_pickle(full_graph, os.path.join(os.pardir, "resources", "basic_graphs", "fullgraph_without_sim.pickle"))

# Mapping the nodes and relabeling them

In [18]:
full_graph = pybel.from_pickle(os.path.join(os.pardir, "resources", "fullgraph_without_sim.pickle"))

In [10]:
relabel_fullgraph = {}
i = 1
for node in full_graph.nodes():
    relabel_fullgraph[node] = i
    i+=1

In [13]:
node_mapping_list = []
protein_list = []
for node, node_id in tqdm(relabel_fullgraph.items()):
    name = node.name
    if node.namespace == 'pubchem':
        name = cid_to_synonyms(node.identifier)
        if not isinstance(name, str):
            name = name.decode("utf-8")
    if node.namespace == 'uniprot':
        protein_list.append(node.identifier)
    node_mapping_list.append((node_id, node.namespace, node.identifier, name))
node_mapping_df = pd.DataFrame(node_mapping_list, columns=['node_id', 'namespace', 'identifier', 'name'])

In [14]:
protein_to_gene = get_gene_names(protein_list)

In [15]:
for protein, gene in protein_to_gene.items():
    node_mapping_df.loc[node_mapping_df['identifier'] == protein, 'name'] = gene

In [16]:
node_mapping_df.head()

,node_id,namespace,identifier,name
0,1,pubchem,85,(3-carboxy-2-hydroxypropyl)-trimethylazanium
1,2,umls,C0000729,Abdominal cramps
2,3,umls,C0000737,Abdominal pain
3,4,umls,C0687713,Gastrointestinal pain
4,5,umls,C0002418,Amblyopia


In [20]:
node_mapping_df.to_csv(os.path.join(os.pardir, "resources", "mapping", "fullgraph_nodes_mapping.tsv"), index=False, sep='\t')

In [18]:
fullgraph_id = nx.relabel_nodes(full_graph, relabel_fullgraph)

In [19]:
nx.write_edgelist(fullgraph_id, os.path.join(os.pardir, "resources", "basic_graphs", "fullgraph_without_sim.edgelist"), data=False)

# Combining the full graph with chemical similarity graph
Note: Calculating chemical similarities and creating the graph is done in a different notebook because it needs the RDkit package and environment

In [44]:
chem_sim_graph = pybel.from_pickle(os.path.join(os.pardir, "resources", "chemsim_50_graphs","chem_sim_graph_50.pickle"))

In [45]:
fullgraph_with_chemsim = full_graph + chem_sim_graph

In [46]:
pybel.to_pickle(fullgraph_with_chemsim, os.path.join(os.pardir, "resources", "chemsim_50_graphs", "fullgraph_with_chemsim_50.pickle"))

In [47]:
chem_sim_relabeled = nx.relabel_nodes(chem_sim_graph, relabel_fullgraph)

In [48]:
nx.write_edgelist(chem_sim_relabeled, os.path.join(os.pardir, "resources", "chemsim_50_graphs","chem_sim_graph_50.edgelist"), data=False)

In [49]:
fullgraph_with_chemsim_relabeled = fullgraph_id + chem_sim_relabeled

In [50]:
nx.write_edgelist(fullgraph_with_chemsim_relabeled, os.path.join(os.pardir, "resources","chemsim_50_graphs", "fullgraph_with_chemsim_50.edgelist"), data=False)